This is a [jupyter](https://jupyter.org/) notebook that contains text, equations, images and executable code in one document.

$
\newcommand{\dd}{\text{d}}
\newcommand{\e}{\operatorname{e}}
\newcommand{\Fc}{\underline F}
\newcommand{\Hc}{\underline H}
\newcommand{\Xc}{\underline X}
\newcommand{\Yc}{\underline Y}
\newcommand{\Zc}{\underline Z}
\newcommand{\xic}{\underline \xi}
\newcommand{\vc}{\underline v}
\newcommand{\pc}{\underline p}
\newcommand{\Ec}{\underline E}
\newcommand{\kc}{\underline k}
\newcommand{\xiv}{\pmb{\xi}}
\newcommand{\Psiv}{\pmb{\Psi}}
\newcommand{\xivc}{\underline \xiv}
\newcommand{\xv}{\mathbf{x}}
\newcommand{\jj}{\operatorname{j}}
$

# 12 Statistical Energy Analysis

## 12.1 Sound Field in a Plate: Statistical Analysis


* from last chapter: assume point load on finite plate
* mode summation
* average over location and over excitation point
\begin{align}
\overline{\overline{v^2}} 
&=\frac{|\Fc_0|^2}{S^2 m''^2}\sum_{N_1}^\infty \sum_{N_2}^\infty
\frac{\omega^2}
{\left(\omega_{N}^2 -\omega^2\right)^2+\eta^2\omega_N^4}
\end{align}
* extra average over frequency
\begin{equation}\label{eq:mean_velo}
\overline{\overline{\overline{v^2}}} \approx\frac{|\Fc_0|^2 \pi}{S^2 m''^2 2\eta\omega}
n= \frac{|\Fc_0|^2 }{S m'' \eta\omega}
 \frac{1}{ 8\sqrt{B'm''}}
\end{equation}
* assume $S=l_x l_z=$const. with aspect ratio $l_x/l_z$
* plot between $\omega_a$ and $\omega_b$
* sum over 1200 modes

In [1]:
import numpy as np
import matplotlib.pyplot as pl
import ipywidgets as ipw
%matplotlib inline

@ipw.interact(eta=(0.01,0.1,0.01),lx=(1,12),
              oma=[0.5,5,50],omb=[50,100,500])
def plot_v2(eta=0.01,lx=4,oma=0.1,omb=50,
            show_modes=False,show_average=False):
    # assume S=12
    lz = 12/lx
    om = np.linspace(oma,omb,1000).reshape((-1,1,1))
    # sum over N1=1...6, N2=1...5 instead of to infinity
    N1 = np.arange(1,lx*10).reshape((1,-1,1))
    N2 = np.arange(1,lz*10).reshape((1,1,-1))
    # assume B1=1 m2=1 F0=1
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    v2 = om**2/lx**2/lz**2/((omN**2-om**2)**2+eta**2 * omN**4)
    # plot mode summands 
    if show_modes:
        v2_ = v2.reshape((v2.shape[0],-1))
        pl.loglog(om[:,0,0],v2_,'g',lw=0.5,label=r'$v^2$')
    # double sum 
    v2 = v2.sum(axis=(1,2))
    # plot
    pl.loglog(om[:,0,0],v2,label=r'$v^2$')
    if show_average:
        pl.loglog(om[:,0,0],1/(lx*lz*eta*om[:,0,0]*8),label=r'$v^2$')
    pl.ylim(1e-5,1e1)
    pl.xlabel(r'$\omega$')
    pl.ylabel(r'$v^2$')

interactive(children=(FloatSlider(value=0.01, description='eta', max=0.1, min=0.01, step=0.01), IntSlider(valu…

* individual modes dominating for low frequencies only 
* frequency average is a good approximation for higher frequency
* approximation gets better with 
  * increasing number of modes per band
  * increasing damping
* result nearly independent of shape for higher frequencies
* energy mean and power mean (from \eqref{eq:mean_velo}, see last chapter):
\begin{align}
P &= \frac{\omega\eta}{2}Sm'' \overline{\overline{\overline{v^2}}}\\
  &= \omega\eta \overline W
\end{align}

* spatial energy distribution from mode summation
* example for point excitation

In [5]:
@ipw.interact(oma=[0.5,5,50],omb=[50,100,500],
              x1=(0,4,0.25),z1=(0,3,0.25),eta=(0.01,0.1,0.01))
def plot_modesum_plate(oma=0.5,omb=500,x1=2.5,z1=1.0,eta=0.01):
    # assume lx=4, lz=3
    lx, lz = 4, 3
    # get coordinates for plot, prepare three extra dims for summation
    x, z = np.meshgrid(np.linspace(0,4,41), np.linspace(0,3,31))
    x = x[:,:,np.newaxis,np.newaxis,np.newaxis]
    z = z[:,:,np.newaxis,np.newaxis,np.newaxis]
    # sum over N1=1...6, N2=1...5 instead of to infinity
    # first two dims are for x,z
    N1 = np.arange(1,40).reshape((1,1,-1,1,1))
    N2 = np.arange(1,30).reshape((1,1,1,-1,1))
    # assume B1=1 m2=1
    om = np.linspace(oma,omb,31).reshape((1,1,1,1,-1))
    # compute each element of the sum in one go
    omN = np.pi**2 * ((N1/lx)**2 + (N2/lz)**2)
    v = 4*om*om/lx/lz *\
        np.sin(np.pi*N1*x/lx)*np.sin(np.pi*N1*x1/lx)* \
        np.sin(np.pi*N2*z/lz)*np.sin(np.pi*N2*z1/lz)/ \
        (omN**2 * (1+1j*eta)-om**2)
    # triple sum N1, N2, om, Dezibel
    v2 = 10*np.log10(1e-6+np.abs(v.sum(axis=(2,3,4)))**2)
    # plot energy on plate
    pl.imshow(v2,cmap='hot_r',origin='lower',
                   vmin=v2.mean()-10,vmax=v2.mean()+30,extent=(0,4,0,3),
             interpolation='bicubic')
    pl.colorbar()

interactive(children=(Dropdown(description='oma', options=(0.5, 5, 50), value=0.5), Dropdown(description='omb'…

* 'diffuse' field, energy evenly distributed, except near the excitation

## 12.2 Two Coupled Plates

![Two point coupled plates](tikzfig/two_plates.svg)
* force and velocity at coupling point: $\Fc_c, \vc_c$
* both plates may have different modes $N, M$ and different masses $m_1,m_2$
* first plate:
\begin{equation}
\vc_1(\xv)=
\frac{4\jj \omega}{m_1}
\sum_{N=1}^{\infty}
\frac{\varphi_N (\xv) \left(\Fc_1\varphi_N (\xv_1)+\Fc_c\varphi_N (\xv_c)\right)}
{\omega_{N}^2 \left(1+\jj \eta\right)-\omega^2}
\end{equation}
* second plate:
\begin{equation}
\vc_2(\xv)=
\frac{4\jj \omega}{m_2}
\sum_{M=1}^{\infty}
\frac{\varphi_M (\xv) \left(\Fc_2\varphi_M (\xv_2)+\Fc_c\varphi_M (\xv_c)\right)}
{\omega_{M}^2 \left(1+\jj \eta\right)-\omega^2}
\end{equation}
* from $\overline{\overline{\overline{v^2_{1/2}}}}$ we get the energy
* coupling:
\begin{equation}
\vc_c=\vc_1(\xv_c)=\vc_2(\xv_c)
\end{equation}
* net power 'flow' from first to second plate
\begin{equation}
P_{12net} = \frac 1 2 \Re\left(\Fc_c\vc_c^*\right)
\end{equation}
* if we just consider one mode from the sum per plate, then after considerable derivation, it turns out that:
\begin{equation}
P_{NMnet} = b_{NM} \left(W_N-W_M\right)
\end{equation}
* power flow is proportional to energy difference between two modes!
* now if all modes in a certain frequency band are taken into account:
\begin{align}
P_{12net} &= \sum_N \sum_M b_{NM} \left(W_N-W_M\right)
\end{align}
* assume equipartition of energy between the modes (all $W_N$ are the same and all $W_M$ are the same)
\begin{gather}
W_1=\Delta N W_N\\
W_2=\Delta M W_M
\end{gather}
where $\Delta N$ and $\Delta M$ are the number of modes in the frequency band $\Delta \omega$ for plate 1 and 2, respectively 
* overall net power flow depends on the energies in the plates 
\begin{align}
P_{12net} &= \frac{1}{\Delta N}\sum_N \sum_M b_{NM} W_1-\frac{1}{\Delta M}\sum_N \sum_M b_{NM} W_2\\
&=\omega\eta_{12}W_1 - \omega\eta_{21}W_2\\
&=P_{12}-P_{21}
\end{align}
where the 'coupling loss factors' $\eta_{12}$ and $\eta_{21}$ were introduced to get the same form as for just one plate
* coupling loss factors are
\begin{gather}
\eta_{12}=\frac{1}{\omega\Delta N}\sum_N \sum_M b_{NM}=\frac{P_{12}}{\omega W_{1}}\\
\eta_{21}=\frac{1}{\omega\Delta M}\sum_N \sum_M b_{NM}=\frac{P_{21}}{\omega W_{2}}
\end{gather}
* consistency or 'reciprocity':
\begin{equation}
n_1\eta_{12}=n_2\eta_{21}
\end{equation}

* coupling loss factor for two point connected plates
\begin{equation}
\eta_{12}\approx\frac{1}{\omega m_1 }\Re\left(\frac{1}{\Zc_1}\right)\frac{\Re(\Zc_1)\Re(\Zc_2)}{\left|\Zc_1 +\Zc_2\right|^2}
\end{equation}
where $\Zc$ is the point impedance of the respective plate
* using the real-valued point impedance for infinite plates, we get
\begin{equation}
\eta_{12}\approx\frac{1}{\omega m_1 }\frac{Z_2}{\left(Z_1 +Z_2\right)^2}
\end{equation}

## 12.3 Generalization: Subsystems

* instead of plates, we could also use beams, shell or any other part of a structure bearing a sound field
* we call this parts then 'subsystems'

![Subsystems](tikzfig/subsystems.svg)
* statistical analysis of the sound field in a subsystem $i$:
  * state of sound field: __stored energy__ $W_i$ is the _expected value_ for the sound energy 
  * excitation (e.g. by external force) is evaluated using the __input power__ $P_i$ (also an expected value)
  * loss of energy (per unit time) is evaluated by the expected value of __power loss__ $P_{ii}$
  * the flux of energy from and to other subsystems $j$ coupled to subsystem $i$ is characterized by the expected value of __coupling power flows__ $P_{ij}$ and $P_{ji}$
* from analysis of a single plate (not coupled) we have
\begin{gather}
P_{i}=\omega\eta_i W_i
\end{gather}
where $\eta_i$ is the damping loss factor in subsystem $i$
* assume $W_i$ does not change over time, then due to conservation of energy $P_{i}=P_{ii}$ and 
\begin{gather}
P_{ii}=\omega\eta_i W_i
\end{gather}
* for subsystems coupled
\begin{gather}
P_{ij}=\omega\eta_{ij} W_i
\end{gather}
* energy 'balance' for each subsystem
\begin{equation}\label{eq:}
P_i=P_{ii}+\sum_j P_{ij} -\sum_j P_{ji}
\end{equation}
* example for three subsystems:

In [6]:
import sympy as sy
eta1,eta2,eta3 = sy.symbols('eta_1:4')
eta12,eta13,eta21,eta23,eta31,eta32 = \
    sy.symbols('eta_12 eta_13 eta_21 eta_23 eta_31 eta_32')
W1,W2,W3 = sy.symbols('W_1:4')
P1,P2,P3 = sy.symbols('P_1:4')
om = sy.symbols('omega')
P11 = om*eta1*W1
P12 = om*eta12*W1
P13 = om*eta13*W1
P22 = om*eta2*W2
P21 = om*eta21*W2
P23 = om*eta23*W2
P33 = om*eta3*W3
P31 = om*eta31*W3
P32 = om*eta32*W3
eq1 = sy.Eq(P1,P11+P12+P13-P21-P31)
eq2 = sy.Eq(P2,P22+P21+P23-P12-P32)
eq3 = sy.Eq(P3,P33+P31+P32-P13-P23)
matr = sy.linear_eq_to_matrix((eq1,eq2,eq3),[W1,W2,W3])
display(-matr[0],-matr[1])

Matrix([
[eta_1*omega + eta_12*omega + eta_13*omega,                             -eta_21*omega,                             -eta_31*omega],
[                            -eta_12*omega, eta_2*omega + eta_21*omega + eta_23*omega,                             -eta_32*omega],
[                            -eta_13*omega,                             -eta_23*omega, eta_3*omega + eta_31*omega + eta_32*omega]])

Matrix([
[P_1],
[P_2],
[P_3]])

* general form for the system of equations
\begin{equation}
\omega \mathbf X \mathbf W= \mathbf P
\end{equation}
where the matrix $\mathbf X$ holds all the loss and coupling loss factors:
\begin{equation}
X_{ij}=-\eta_{ji},\;\; i \neq j\\
X_{ii}=\eta_i+\sum_{j\neq i}\eta_{ij}
\end{equation}
* $\mathbf X$ can be rendered symmetric if instead of the total energies $W_i$ the energies per mode $\frac{W_i}{n_i}$ are used
* simple example: three point-connected plates of same thickness and size

![three plates, where plate 1 is connected to plate 2 and plate 2 to plate 3](tikzfig/three_plates.svg)

* excitation only at plate 1
* point impedance and mass are the same for all three plates
* damping is also the same

In [7]:
# frequency
numfreq = 31 # number of frequencies
om = np.logspace(0,3,numfreq)
# assume m2=1, B1=1
Z = 8
# assume S=1, thus m=1
etaij = 1/om/(4*Z) # etaij=etaji, because n1=n2

@ipw.interact(etai = (0.01,0.1,0.01))
def plot_three_plates(etai = 0.01):
    # excitation only in plate 1 with power 1
    P = np.zeros((numfreq,3))
    P[:,0] = 1
    # assemble coupling loss factor matrix for all frequencies
    X = np.zeros((numfreq,3,3))
    # diagonal elements
    X[:,0,0] = etai+etaij
    X[:,1,1] = etai+2*etaij
    X[:,2,2] = etai+etaij
    # off-diagonal elements
    X[:,0,1] = -etaij
    X[:,1,0] = -etaij
    X[:,1,2] = -etaij
    X[:,2,1] = -etaij
    # solve for all frequencies
    W=np.linalg.solve(om[:,np.newaxis,np.newaxis]*X,P)
    # plot energies ()
    pl.loglog(om,W[:,0],label=r'$W_1$')
    pl.loglog(om,W[:,1],label=r'$W_2$')
    pl.loglog(om,W[:,2],label=r'$W_3$')
    pl.legend()

interactive(children=(FloatSlider(value=0.01, description='etai', max=0.1, min=0.01, step=0.01), Output()), _d…

## 12.4 Coupling Loss Factors

* different approaches to estimate coupling loss factors
  * modal summation approach (see above, e.g. plate-plate point connections)
  * radiation approach (e.g. beam-plate line connection)
  * traveling wave approach (e.g. plate-plate line connection)
  
#### Traveling wave approach
* equipartition of energy and mode summation is equivalent to diffuse field
* (infinitely) many plane waves carry energy in all directions with same probability and intensity
* interface: line connection with length $l$
* power incident from direction $\vartheta$:
\begin{equation}
P_e = c_{g}W'' l \cos(\vartheta)
\end{equation}
where $W''$ is the infinitesimal energy density in the plane wave
* integrate over all wave directions and the plate area to get the overall energy 
\begin{equation}
W_i = \int_S \int_{-\pi/2}^{\pi/2} W'' \dd \vartheta \dd S = \pi W''S
\end{equation}
* integrate over all wave directions to get energy flux (power flow) over the interface as given by the transmission coefficient
\begin{equation}
P_{ij}=c_{g}W'' l \int_{-\pi/2}^{\pi/2}\cos(\vartheta)\tau(\vartheta) \dd \vartheta
\end{equation}
* coupling loss factor 
\begin{equation}
\eta_{ij} = \frac{P_{ij}}{\omega W_i}=\frac{2 c_g l}{\pi \omega S}\overline{\tau}
\end{equation}
where
\begin{equation}
\overline{\tau}=\int_{0}^{\pi/2}\cos(\vartheta)\tau(\vartheta) \dd \vartheta=\int_0^1 \tau(\sin(\vartheta))\, \dd \sin(\vartheta)
\end{equation}
* we can use the approach from chapter [Transmission over Plate Junctions](./08_Plate_Junctions.ipynb)


#### ----------------------------------------
* wave field in finite plates can be analyzed statistically
* diffuse field assumptions
* statistical description of interaction of sound fields of two coupled plates
* coupling loss factors
* subsystem and energy balance
* relation to transmission coefficients
* next topic: application of statistical energy analysis

#### License

This notebook is an [Open Educational Resource](https://en.wikipedia.org/wiki/Open_educational_resources). Feel free to use it for your own purposes. The text and the images are licensed under [Creative Commons Attribution 4.0](https://creativecommons.org/licenses/by/4.0/), and any code under the [MIT license](https://opensource.org/licenses/MIT). Please attribute the work as follows: Ennes Sarradj, Structure-borne sound lecture notes, 2019.

In [5]:
# this is just for custom formatting
from IPython.core.display import HTML
def css_styling():
    styles = open("./custom.css", "r").read()
    return HTML(styles)
css_styling()